Libraries

In [5]:
#!/usr/bin/env python3

import rospy
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import OccupancyGrid
import numpy as np

Map Parameters

In [2]:
map_width = 100 # meters
map_height = 100 # meters
map_resolution = 0.1 # meters
scan_range = 8 # meters

Scan Information

In [14]:
def scan_data(data):
    global occupancy_grid
    # initialise occupancy grid
    occupancy_grid = OccupancyGrid()
    occupancy_grid.header.frame_id = "map"
    occupancy_grid.header.stamp = rospy.Time.now()
    occupancy_grid.info.resolution = map_resolution
    occupancy_grid.info.width = int(map_width / map_resolution)
    occupancy_grid.info.height = int(map_height / map_resolution)
    occupancy_grid.info.origin.position.x = -map_width/2
    occupancy_grid.info.origin.position.y = -map_height/2
    occupancy_grid.data = [-1] * occupancy_grid.info.width * occupancy_grid.info.height

    # convert scan data to occupancy grid data
    for i, scan_value in enumerate(data.ranges):
        if scan_value < scan_range:
            # convert angle to index in the occupancy grid array
            angle = data.angle_min + i * data.angle_increment
            x = int((scan_value * np.cos(angle) + map_width/2) / map_resolution)
            y = int((scan_value * np.sin(angle) + map_height/2) / map_resolution)
            # set the corresponding cell in the occupancy grid to occupied
            occupancy_grid.data[x + y * occupancy_grid.info.width] = 100

    # publish the occupancy grid
    occupancy_grid_pub.publish(occupancy_grid)

if __name__ == '__main__':
    rospy.init_node('occupancy_grid_node', anonymous=True)
    rospy.Subscriber("/scan", LaserScan, scan_data)
    occupancy_grid_pub = rospy.Publisher("/map", OccupancyGrid, queue_size=10)

    rospy.spin()    
    # rate = rospy.Rate(10)
    # while not rospy.is_shutdown():
    #     rate.sleep()